<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

In [2]:
#загрузим данные
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
#предварительно оценим данные
display(df.head(5))
print()
display(df.info())
print()
display(df.describe())
print()
display(df.isna().sum())
print()
print(df.duplicated().sum())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


153


In [4]:
#заменим тип данных в колонках Возраст и Зарплата (дробной части все равно нет)
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')
#проверим
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


None

In [5]:
#интересно посмотреть на гендерное распределение застрахованных 
display(df['Пол'].value_counts(normalize=True))
    

0    0.501
1    0.499
Name: Пол, dtype: float64

In [6]:
#оценим кореляцию признаков
#df.corr()

Вывод п.1. Данные загружены и хорошего качества. Пропусков нет, выбросов и прочих аномалий не наблюдаем. Дубликаты не трогали - это могут быть совпадающие признаки клиентов. Тип данных в столбцах Возраст и Зарплата заменили на целочисленный. Наблюдаются почти равные категории клиентов (мужчин и женщин). Средний возраст застрахованного составляет около 31 года и почти равен медиане в 30 лет. Ограничения возраста застрахованных от 18 до 65 лет определяется вероятно законодательно, компания не занимается страхованием лиц, чей возраст составляет менее 18 и старше 65. Общее впечатление, что датасет заранее подготовлен - внесенные мной изменения очень незначительны.

## Умножение матриц

Проверка изменения качества линейной регрессии

In [7]:
#выделим признаки и целевой признак
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

features.shape, target.shape

((5000, 4), (5000,))

In [8]:
#формула линейной регрессии с единичным столбцом
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1) 
y = target

#посчитаем параметр w по формуле минимизации MSE
w = np.linalg.inv(X.T @ X) @ X.T @ y          
print('параметр w:            ', w[1:])

#модель
model = LinearRegression()

#обучение модели
model.fit(features, target)

#оценочные коэффициенты для линейной регрессии
print('оценочные коэффициенты:', model.coef_) 

параметр w:             [ 7.92580563e-03  3.57083050e-02 -1.70081903e-07 -1.35676627e-02]
оценочные коэффициенты: [ 7.92580563e-03  3.57083050e-02 -1.70081903e-07 -1.35676627e-02]


Обозначения для работы с уравнениями линейной регрессии:
    
 $X$ — матрица признаков (нулевой столбец состоит из единиц)
    
 $y$ — вектор целевого признака
    
 $P$ — матрица, на которую умножаются признаки
    
 $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)
   
 Предсказания: $a = Xw$
   
 Задача обучения: $w = argmin_w MSE(Xw, y)$
    
 Формула обучения: $w = (X^T X)^{-1} X^T y$
    
 **Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
    
 **Ответ:** не изменится.
    
 **Обоснование:** $a = X_1w$
 
В формулу вектора весов линейной регрессии $w$ подставляем произведение $XP$ вместо $X$. Получится выражение для $w_1$ (вектор весов после преобразования). $P$ – обратимая квадратная матрица (матрица преобразования). Используем вышеприведенные формулы, преобразуем получившееся выражение и получаем зависимость между параметром $w_1$ (вектор весов после преобразования) и $w$ (вектор весов до преобразования). Затем в формулу предсказания $а = Xw$ подставляем $XP$ вместо $Х$, и $w_1$ вместо $w$.

**Обоснование:**

Запишем исходное уравнение весов: $w = (X^T X)^{-1} X^T y$

Докажем, что при умножении на матрицу имеющую обратную ничего не изменится. Запишем вместо $Х$ произведение $ХР$, тогда:

$w_1 = ((XP)^T(XP))^{-1}(XP)^Ty$ = $(P^TX^TXP)^{-1}P^TX^Ty$ = $P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty$, где $(P^T)^{-1}P^T = E$, тогда $w_1 = P^{-1}((X^TX)^{-1}X^Ty)$ = $P^{-1}w$, 
мы нашли связь между $w$ и $w_1$, подставим в $a = Xw:$

**Получаем:** $a_1 = XPw_1 = XPP^{-1}w = XEw = Xw = a$

 
    
**Доказано**, что преобразование подобного типа не повлияет на предсказание.

Промежуточный вывод. Оценочные коэффициенты и параметр w полностью совпадают, качество линейной регрессии не изменяется.

In [9]:
#считаем r2 для модели
print('r2 для исходных признаков:  ', r2_score(y, model.predict(features))) 

#генерируем случайную матрицу
random = np.random.normal(size=(4,4))

#проверка матрицы на обратимость
np.linalg.inv(random) 

#изменяем матрицу признаков 
features_inv = features @ random 

#модель для измененных признаков
lr = LinearRegression()

#обучение модели
lr.fit(features_inv, target)

#считаем r2 для измененных признаков
print('r2 для измененных признаков:', r2_score(target, lr.predict(features_inv))) 

r2 для исходных признаков:   0.42494550308169177
r2 для измененных признаков: 0.4249455030817183


Промежуточный вывод. Линейная регрессия работает. r2 совпадают с точностью до 14 знаков и 0<r2<1. 

Вывод п.2. Параметр w и оценочные коэффициенты, оценки r2 в исходной задаче и преобразованной совпадают, следовательно, качество линейной регрессии не изменяется.

## Алгоритм преобразования

Мы меняли признаки с помощью случайно сгенерированной матрицы. Я предлагаю следующий алгоритм преобразования данных для защиты информации:

1. Генерирую случайную квадратную матрицу
2. Умножаю матрицу признаков на случайную квадратную матрицу и, например, на константу PI = 3.14

In [10]:
PI = 3.14

#случайная матрица
random = np.random.normal(size=(4,4))

#проверим на обратимость
np.linalg.inv(random)

#шифруем данные
features_inv = features @ random * PI 

#посмотрим на результат
features_inv.head(5)

,0,1,2,3
0,-112830.323715,233982.518947,30280.447734,-73353.602513
1,-86480.184065,179320.830097,23079.067197,-56150.477665
2,-47802.396799,99112.692388,12729.292166,-31020.362387
3,-94817.788253,196654.257644,25558.302786,-61711.095431
4,-59392.427750,123153.382370,15885.592899,-38579.815789


In [11]:
#модель
lr = LinearRegression() 

#обучение модели
lr.fit(features_inv, target) 

#считаем r2 для измененных признаков
print('r2:', r2_score(target, lr.predict(features_inv))) 

r2: 0.42494550308169154


Выводы п.3. Алгоритм справляется с защитой данных -  r2 защищенных данных совпадает с r2 исходных данных. 

## Проверка алгоритма

Декодируем признаки и сравним с исходными

In [12]:
print('Исходные признаки:')

#посмотрим исходные признаки
display(features.head(5)) 

print()
decode = round(features_inv @ np.linalg.inv(random) * (1/PI)).astype(int)

#заменим названия колонок для наглядности
decode.columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи'] 
print('Декодированные признаки:')

#посмотрим декодированные признаки
display(decode.head(5)) 

Исходные признаки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0



Декодированные признаки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Вывод п.4. Проверено декодирование признаков - алгоритм работает. 

Общий вывод. В ходе проекта были загружены и изучены данные о клиентах страховой компании. Установлено, что данные хорошего качества и практически готовы к работе. Поменяли тип данных в 2-х колонках. Были рассчитаны параметр w и оценочные коэффициенты. Рассчитаны оценки r2 в исходной и преобразованной задаче, значения совпали, что говорит о неизменном качестве линейной регрессии. Был предложен алгоритм преобразования (шифрования) данных. Установлено, что алгоритм справляется с защитой данных - r2 защищенных данных совпадает с r2 исходных данных. Проверено декодирование признаков с помощью обратной операции - данные совпадают с исходными. 
Результатом работы над проектом является опыт практической защиты персональных данных, используя матричные операции.